In [6]:
# OBJECTIVE
# To control the shower temperature and keep it between 37 and 39 °C

# PARAMETERS
# Shower length: 60 seconds
# Actions: Turn Down, Turn Up, Leave It
# Task: Build a model that keeps us in the optimal range for as long as possible

# UNCOMMENT THIS BEFORE RUNNING THE PROGRAM
# !pip install tensorflow==2.3.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2

from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [7]:
class ShowerEnv(Env):
    
    def __init__(self):
        # Initialize constants
        self.starting_temp = 38
        self.temp_deviation = 3
        self.num_actions = 3
        self.temp_low = 37
        self.temp_high = 39
        self.temp_noise = 1
        self.max_shower_length = 60
        
        # Actions we can take: up, down, or stay
        self.action_space = Discrete(self.num_actions)
        
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        
        # Set start temperature
        self.state = self.starting_temp + random.randint(-self.temp_deviation, self.temp_deviation)
        
        # Set shower length
        self.shower_length = self.max_shower_length
    #end
    
    
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0
        # 2 -1 = 1 temperature
        self.state += action - 1
        
        # Reduce shower length by 1 second
        self.shower_length -= 1
        
        # Calculate reward
        if self.state >= self.temp_low and self.state <= self.temp_high:
            reward = 1
        else:
            reward = -1
        #end
        
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        #end
        
        # Apply temperature noise
#         self.state += random.randint(-self.temp_noise, self.temp_noise)
        
        # Set placeholder
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    #end
    
    
    def render(self):
        pass
    #end
    
    
    def reset(self):
        # Reset shower temperature
        self.state = self.starting_temp + random.randint(-self.temp_deviation, self.temp_deviation)
        
        # Reset shower time
        self.shower_length = self.max_shower_length
        
        return self.state
    #end
    
#end

In [8]:
env = ShowerEnv()

In [9]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    #end
    print("Episode: {} Score: {}".format(episode, score))
#end

Episode: 1 Score: -34
Episode: 2 Score: -38
Episode: 3 Score: 12
Episode: 4 Score: -24
Episode: 5 Score: -48
Episode: 6 Score: 22
Episode: 7 Score: -52
Episode: 8 Score: -34
Episode: 9 Score: -60
Episode: 10 Score: -42


In [10]:
states = env.observation_space.shape
actions = env.action_space.n

In [11]:
actions

3

In [12]:
# Create a deep learning model with Keras

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

def build_model(states, actions):
    model = Sequential()
    #model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
#end

2021-09-03 11:34:22.143512: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-09-03 11:34:22.143552: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [65]:
del model
model = build_model(states, actions)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 24)                48        
_________________________________________________________________
dense_16 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [66]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [67]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn
#end

In [68]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 55s 6ms/step - reward: -0.5118
166 episodes - episode_reward: -30.614 [-60.000, 34.000] - loss: 1.130 - mae: 6.805 - mean_q: -9.533

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 55s 5ms/step - reward: -0.4970
167 episodes - episode_reward: -29.856 [-60.000, 28.000] - loss: 2.375 - mae: 10.876 - mean_q: -15.591

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: -0.4038
167 episodes - episode_reward: -24.299 [-60.000, 34.000] - loss: 2.390 - mae: 10.965 - mean_q: -15.735

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 64s 6ms/step - reward: -0.3628
166 episodes - episode_reward: -21.614 [-60.000, 42.000] - loss: 2.033 - mae: 9.936 - mean_q: -14.208

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 61s 6ms/step - reward: -0.35

In [69]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: -60.000, steps: 60
Episode 4: reward: -58.000, steps: 60
Episode 5: reward: -60.000, steps: 60
Episode 6: reward: -54.000, steps: 60
Episode 7: reward: -56.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -54.000, steps: 60
Episode 12: reward: -56.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -54.000, steps: 60
Episode 16: reward: -54.000, steps: 60
Episode 17: reward: -54.000, steps: 60
Episode 18: reward: -56.000, steps: 60
Episode 19: reward: -56.000, steps: 60
Episode 20: reward: -54.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: -54.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: -60.000, steps: 60
Episode 25: reward: -54.000, steps: 60
Episo